Implementation of policy iteration (Sutton and Barto, section 4.3, page 80)

In [2]:
import pandas as pd
import numpy as np

In [331]:
class Agent:

    '''
    Initializes the agent
    @param rows - The number of rows in the grid world
    @param columns - The number of columns in the grid world
    @param terminal_state - The final state the agent is trying to find the best path to
    '''
    def __init__(self, rows, columns, terminal_state, theta=0.05, gamma=0.05):
        # self.grid = np.random.rand(rows, columns) * -1 # Initializes
        self.rows = rows
        self.columns = columns
        self.values = np.zeros((rows, columns)) # Initializes all state values to zero
        self.policies = self.initializePolicies()
        # self.policies = self.initializePolicies() # Initializes policies for each cell,
        #                                           # policies[row][column] is an array where the value
        #                                           # at index 0 is the probability that up is the optimal choice,
        #                                           # 1 is right, 2 is down, 3 is left (clockwise around Cartesian)
        self.terminal_state = terminal_state # Where the end state is located on the grid
        self.theta = theta # A value close to zero signifying completion, determines the accuracy of the policy estimation
        self.gamma = gamma # A value signifying by how much to discount future rewards 

    '''
    Initializes the policies for each cell in the grid world
    '''
    def initializePolicies(self):
        policies = np.empty((self.rows, self.columns, 4))

        # Fill the array with random values that sum to 1
        for i in range(self.rows):
            for j in range(self.columns):
                policies[i, j] = np.zeros(4)
                policies[i, j][np.random.randint(4)] = 1

        return policies

    def isValidState(self, coords):
        return coords[0] >= 0 and coords[0] < self.rows and coords[1] >= 0 and coords[1] < self.columns

    '''
    Gets the available successor states (necessary because our grid world has walls)
    @param coords - coordinates in tuple form (x, y) of our state
    @return a list of successors
    '''
    def getAvailableSuccessorStates(self, coords):
        successors = []
        
        # if coords[1] != 0:
        successors.append((coords[0], coords[1] - 1))

        # if coords[0] != self.rows - 1:
        successors.append((coords[0] + 1, coords[1]))

        # if coords[1] != self.columns - 1:
        successors.append((coords[0], coords[1] + 1))

        # if coords[0] != 0:
        successors.append((coords[0] - 1, coords[1]))

        return successors

    def getDiscountedValuesForSuccessors(self, state):
        values = np.zeros(4)
        
        successors = self.getAvailableSuccessorStates(state[0]) # Get possible next states
                
        value = 0
        for successor in range(len(successors)):
            if self.isValidState(successors[successor]):
                probability_successor_chosen = self.policies[state[0][0]][state[0][1]][successor]
                successor_value = self.getReward(state[0]) + self.gamma * self.values[state[0][0]][state[0][1]]
                values[successor] = probability_successor_chosen * successor_value
        
        return values

    def getReward(self, state):
        if state[0] == self.terminal_state[0] and state[1] == self.terminal_state[1]:
            return 100
        return 0
        

    '''
    Performs a policy evaluation step, updates the policy
    '''
    def policyEvaluation(self):
        while True:
            delta = 0 # This is the check to know when to stop evaluation
            for state in np.ndenumerate(self.values): # Iterate over states - in the form ((row, column), value)
                old_value = self.values[state[0]] # Get value of the state
                
                value = np.sum(self.getDiscountedValuesForSuccessors(state))
                self.values[state[0]] = value

                delta = max(delta, np.absolute(old_value - value))
            
            if delta < self.theta:
                break

    def policyImprovement(self):
        while True:
            stable = True
            for state in np.ndenumerate(self.values):
                old_action = np.argmax(self.policies[state[0]])
                # print(self.getDiscountedValuesForSuccessors(state))
                self.policies[state[0]] = self.getDiscountedValuesForSuccessors(state)
                action = np.argmax(self.policies[state[0]])
                if old_action != action:
                    stable = False
            
            if not stable:
                # print(self.policies)
                self.policyEvaluation()
            else:
                break

In [332]:
a = Agent(3, 5, (2, 4))

In [333]:
a.values

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [334]:
a.policies

array([[[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.]]])

In [335]:
a.policyImprovement()

In [336]:
a.values

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [337]:
a.policies

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])